In [ ]:
# ===== NOTEBOOK: 01_extraccion.ipynb =====
# Sistema Híbrido de Extracción de Datos
# Yahoo Finance + Alpha Vantage

# %% [markdown]
# # 📊 Extracción de Datos Financieros - Sistema Híbrido
# 
# Este notebook usa un sistema híbrido que combina:
# 1. **Yahoo Finance** (base, gratis, ilimitado)
# 2. **Alpha Vantage** (complemento, 500 calls/día gratis)
# 
# ## 🎯 Objetivo
# Extraer datos para **1 empresa** (AAPL) como prueba antes de escalar a 3000 empresas

# %%
# Imports
import sys
import os
sys.path.append('../')

from src.data_collector import HybridDataCollector, collect_single_company
from config.api_keys import ALPHA_VANTAGE_KEY, USE_HYBRID_MODE

import pandas as pd
import numpy as np

print("✅ Módulos importados correctamente")

# %% [markdown]
# ## 🔧 Configuración

# %%
# Ticker para la prueba
TICKER = "AAPL"

# Rango de fechas
START_DATE = "2016-01-01"
END_DATE = "2023-12-31"

# Modo de operación
USE_HYBRID = False  # Cambiar a True cuando tengas API key de Alpha Vantage

print(f"🎯 Configuración:")
print(f"   Ticker: {TICKER}")
print(f"   Período: {START_DATE} a {END_DATE}")
print(f"   Modo híbrido: {'✅ Activado' if USE_HYBRID else '❌ Solo Yahoo Finance'}")

# %% [markdown]
# ## 📥 OPCIÓN 1: Recolección Simple (Una Empresa)

# %%
# Forma más simple - usando función helper
print(f"\n{'='*60}")
print(f"🚀 Iniciando recolección para {TICKER}")
print(f"{'='*60}\n")

result = collect_single_company(
    ticker=TICKER,
    use_hybrid=USE_HYBRID,
    alpha_vantage_key=ALPHA_VANTAGE_KEY if USE_HYBRID else None
)

print(f"\n✅ Recolección completada")
print(f"\nCalidad de datos:")
print(f"   - Datos de mercado: {'✅' if result['data_quality']['has_market_data'] else '❌'}")
print(f"   - Datos financieros: {'✅' if result['data_quality']['has_financial_data'] else '❌'}")
print(f"   - Shares outstanding: {'✅' if result['data_quality']['has_shares'] else '❌'}")
print(f"   - Score total: {result['data_quality']['overall_score']:.0f}%")

# %% [markdown]
# ## 📥 OPCIÓN 2: Recolección Detallada (Control Total)

# %%
# Si necesitas más control, usa la clase directamente
collector = HybridDataCollector(
    alpha_vantage_key=ALPHA_VANTAGE_KEY if USE_HYBRID else None,
    use_hybrid=USE_HYBRID
)

# Recolectar
result_detailed = collector.collect_company_data(
    ticker=TICKER,
    start=START_DATE,
    end=END_DATE
)

# Guardar
collector.save_data(result_detailed, output_dir="../data/raw")

# %% [markdown]
# ## 🔍 Inspeccionar Datos Recolectados

# %%
# Ver datos de mercado
if result['market_data'] is not None:
    print("\n📈 DATOS DE MERCADO")
    print(f"   Filas: {len(result['market_data'])}")
    print(f"   Columnas: {list(result['market_data'].columns)}")
    print(f"\n   Muestra (últimos 5 días):")
    display(result['market_data'].tail())

# %%
# Ver datos financieros
if result['financial_data'] is not None:
    print("\n📊 DATOS FINANCIEROS")
    print(f"   Reportes: {len(result['financial_data'])}")
    print(f"   Columnas: {len(result['financial_data'].columns)}")
    print(f"\n   Primeras 10 columnas:")
    print(result['financial_data'].columns[:10].tolist())
    print(f"\n   Muestra (últimos 3 reportes):")
    display(result['financial_data'].tail(3))

# %%
# Ver columnas disponibles
if result['financial_data'] is not None:
    print("\n📋 TODAS LAS COLUMNAS DISPONIBLES:")
    for i, col in enumerate(result['financial_data'].columns, 1):
        print(f"{i:3d}. {col}")

# %% [markdown]
# ## 🔍 Verificar Disponibilidad de Variables Críticas

# %%
from src.diccionario_variables import check_data_availability, VARIABLES_CRITICAS

if result['financial_data'] is not None:
    available, missing = check_data_availability(result['financial_data'].columns)
    
    print(f"\n✅ VARIABLES DISPONIBLES ({len(available)}):")
    for var in available:
        print(f"   • {var}")
    
    if missing:
        print(f"\n❌ VARIABLES FALTANTES ({len(missing)}):")
        for var in missing:
            print(f"   • {var}")

# %% [markdown]
# ## 💾 Verificar Archivos Guardados

# %%
import os

raw_dir = "../data/raw"
files = [f for f in os.listdir(raw_dir) if TICKER in f]

print(f"\n📂 Archivos guardados para {TICKER}:")
for file in sorted(files):
    file_path = os.path.join(raw_dir, file)
    file_size = os.path.getsize(file_path) / 1024  # KB
    print(f"   • {file} ({file_size:.1f} KB)")

# %% [markdown]
# ## 🎯 Próximos Pasos
# 
# 1. ✅ Datos extraídos correctamente
# 2. ➡️ Continuar con `02_limpieza_mapeo.ipynb` para procesar los datos
# 3. ➡️ Luego `03_ingenieria_factores.ipynb` para calcular características

# %% [markdown]
# ## 🚀 BONUS: Preparación para Múltiples Empresas
# 
# Cuando estés listo para escalar a 3000 empresas, usa este código:

# %%
"""
# NO EJECUTAR AHORA - Solo para referencia futura

tickers = ['AAPL', 'MSFT', 'GOOGL', ...]  # Lista de 3000 tickers

collector = HybridDataCollector(
    alpha_vantage_key=ALPHA_VANTAGE_KEY,
    use_hybrid=True
)

results = []
for i, ticker in enumerate(tickers, 1):
    print(f"\\n[{i}/{len(tickers)}] Procesando {ticker}...")
    
    try:
        result = collector.collect_company_data(ticker)
        collector.save_data(result)
        results.append(result)
        
        # Cada 100 empresas, hacer checkpoint
        if i % 100 == 0:
            print(f"✅ Checkpoint: {i} empresas procesadas")
            
    except Exception as e:
        print(f"❌ Error con {ticker}: {e}")
        continue

print(f"\\n✅ Completado: {len(results)} empresas procesadas")
"""